# CJ  - DATA EXTRACTING

브랜드별 카드소비 점유율? 브랜드마다 카드점유율을 보여달라!

### [전체과정]
1. 데이터가져오기 (RETREIVE MONTHLY DATA)

 - (where) exclude : negative payment 
 - revisit_his : addr, card_name, cnt_pay
 - inner join : company_id(co_name, cate, cate1)
 * cardname, co_name, addr - refining required
 
2. 전처리

 - 정제<BR/>
     1) 브랜드의 카드점유율을 보려면 payment가 -0인 데이터를 제거해야 한다. <BR/>
        ( 중복된 데이터는 카드점유율을 과대추산한다.)<BR/>
     (ex) 0보다 작은 payment 제거 2832826 -> 2734284

 - 필요한 카테고리(브랜드)에 해당하는 데이터만 추출하기(MERGE & PICKout)<BR/>
    1) 카드사별 분류가 필요함. 정규식적용 <br/>
    2) 브랜드별 정규식 적용.<br/>
    3) 브랜드 x 카드사 의 matrix 생성<br/>
 
3. 브랜드별/카드사별 데이터 count하기(SUM+GROUP BY)
4. 그래프로 시각화하기



In [1]:
from sqlalchemy import create_engine
import pymysql, pandas as pd
pymysql.install_as_MySQLdb()
import MySQLdb,pyarrow

# 1. Retreive data

### REVISIT HIS DATA - 우선순위 1번째

In [2]:
p1= pd.read_pickle('data/revisit_his_p1.pick')
p2= pd.read_pickle('data/revisit_his_p2.pick')
p3= pd.read_pickle('data/revisit_his_p3.pick')
p4= pd.read_pickle('data/revisit_his_p4.pick')
p5= pd.read_pickle('data/revisit_his_p5.pick')

data = pd.concat([p1,p2,p3,p4,p5])
data['PAYMENT']=data['PAYMENT'].astype(int)
print(data.shape," would be reduced to ",data[data['PAYMENT']>0].shape)
print(data.drop_duplicates('COMPANY_ID').shape)
data.head()

(2832826, 4)  would be reduced to  (2734284, 4)
(458269, 4)


,EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT
0,eppcn15a40d5c3cb6ada7e6ac607ec,5940135,KB국민카드(3*4*),11980
1,eppcn159a64276cb6ada7762f7ee01,5791118,신한카드(5*8*),6500
2,eppcn15a250c93cb6ada272ce5e77e,14366,현대카드ZERO,12000
3,eppcn159f69615cb6ada0c9b259456,5434086,KB국민카드(1*1*),61200
4,eppcn15a1512d6cb6ada72bfb2273d,7112451,KB국민카드(9*5*),4500


### COMPANY INFO  - 우선순위 1번째

In [3]:
company=pd.read_pickle('data/company_true.pick')
print(company.shape)

(458268, 5)


# 2. Preprocessing 


## Refine
 - TAKE DATA with the condition to the column, "PAYMENT", which has nonnegative values  
    the purpose of this coding is to get what portion of market-share each brand takes !
    Hence, the canceled data should not be counted becaue the customer having used that store just tried "ONCE" to buy a product of.
 - Clean cardname data

In [4]:
# data[:100].to_parquet('data/test')
data =data[data['PAYMENT']>0]
data.shape
data.head(3)

,EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT
0,eppcn15a40d5c3cb6ada7e6ac607ec,5940135,KB국민카드(3*4*),11980
1,eppcn159a64276cb6ada7762f7ee01,5791118,신한카드(5*8*),6500
2,eppcn15a250c93cb6ada272ce5e77e,14366,현대카드ZERO,12000


In [5]:
def cleanCardName(sentence):
    import re
    pattern = re.compile(r'\(.{0,4}\)|re\)|\<현장즉시결제\>|\[|\s|카드|\<|\>|\'"')
    sentence = re.sub(pattern,'',sentence.lower())
    return sentence # return 안쓰면 none값 뜬다.
    
data = data.assign( CARD_NAME = data['CARD_NAME'].apply(cleanCardName) )


## Merge : revisit + company
 for getting Store information (name, cate, addr)

In [6]:
data_comp = pd.merge(data,company[company.BRAND!='na'],how='inner',on='COMPANY_ID')
print(data_comp.shape)
data_comp.head(3)

(142530, 8)


,EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT,CO_NAME,CATE,CATE1,BRAND
0,eppcn15a2ab595cb6ada272ce5ed16,6631842,농협bc,7000,휴,카페/디저트,카페,커피기타
1,eppcn15a0115e6cb6ada0c9b259b5e,306820,현대thered,6700,롯데시네마광명아울렛,문화,문화,롯데시네마
2,eppcn15a4a2ea2cb6ada16b6ab50df,306820,현대,8000,롯데시네마광명아울렛,문화,문화,롯데시네마


## Merge : brands

In [7]:
brand= pd.read_excel('BRANDCODE.xlsx')
brand.columns=['GROUP','GRUOP_NAME','CODE','CODE_NAME','SYN','STOP']
data_comp_b = data_comp.merge(brand,how='inner',left_on='BRAND',right_on='CODE_NAME')[['EPOPCON_ID','CARD_NAME','BRAND','PAYMENT','GROUP','CODE']]
print(data_comp_b.shape)
data_comp_b.head(3)

(142530, 6)


,EPOPCON_ID,CARD_NAME,BRAND,PAYMENT,GROUP,CODE
0,eppcn15a2ab595cb6ada272ce5ed16,농협bc,커피기타,7000,2,999
1,eppcn15a08e902cb6ada0c9b259fc8,신한,커피기타,69800,2,999
2,eppcn15a46952a8c0c241b8a0194ae,삼성,커피기타,219000,2,999


### SUM/GROUP_BY + RENAME_COLUMNS

 - 영화, 커피, 화장품 별로 데이터나누기

In [8]:
movie = data_comp_b[data_comp_b['GROUP']==1]
coffee = data_comp_b[data_comp_b['GROUP']==2]
beauty = data_comp_b[data_comp_b['GROUP']==4]
print(data_comp_b.shape, movie.shape, coffee.shape, beauty.shape)#(142530, 5) (12033, 5) (109292, 5) (21205, 5)
beauty.head()

(142530, 6) (12033, 6) (109292, 6) (21205, 6)


,EPOPCON_ID,CARD_NAME,BRAND,PAYMENT,GROUP,CODE
93512,eppcn159fb0996cb6ada0c9b2597cd,삼성,올리브영,22000,4,0
93513,eppcn159fb0996cb6ada0c9b2597cd,삼성,올리브영,22000,4,0
93514,eppcn159fb0996cb6ada0c9b2597cd,삼성,올리브영,22000,4,0
93515,eppcn1596bc35ecb6ada77a7319355,하나,올리브영,8000,4,0
93516,eppcn159c0fafdcb6ada7762f7efea,삼성,올리브영,19560,4,0


In [14]:
import gc
gc.collect()
def make_stat_df(df):
    p_stat = df[['BRAND','PAYMENT']].groupby(['BRAND']).agg(['count','sum']).reset_index()
    p_stat.columns = ['CODE_NAME','CNT_PAY','SUM_PAY'] 
    p_stat['TOTAL_CNT'] = sum(p_stat['CNT_PAY'])
    p_stat['TOTAL_SUM'] = sum(p_stat['SUM_PAY'])
    
    p_stat = p_stat.assign(PCT_CNT=p_stat.apply(lambda x: round(x['CNT_PAY']/x['TOTAL_CNT'],4)*100, axis=1))
    p_stat['PCT_SUM'] = p_stat.apply(lambda x: round(x['SUM_PAY']/x['TOTAL_SUM'],4)*100, axis=1)
    return p_stat[['CODE_NAME','PCT_CNT','PCT_SUM']]
# make_stat_df(movie).head(52)
make_stat_df(coffee).head(52)
# make_stat_df(beauty).head(52)

,CODE_NAME,PCT_CNT,PCT_SUM
0,스타벅스,10.42,11.84
1,엔제리너스,1.84,1.41
2,이디야,7.61,3.95
3,카페베네,1.02,0.82
4,커피기타,68.13,72.97
5,커피빈,1.31,1.05
6,탐앤탐스,1.27,0.93
7,투썸플레이스,4.80,4.30
8,파스쿠찌,1.43,1.15
9,할리스,2.17,1.59


In [15]:
brand[['GROUP','GRUOP_NAME','CODE','CODE_NAME']].merge(make_stat_df(coffee),how='inner',on='CODE_NAME')

,GROUP,GRUOP_NAME,CODE,CODE_NAME,PCT_CNT,PCT_SUM
0,2,커피,0,투썸플레이스,4.80,4.30
1,2,커피,1,스타벅스,10.42,11.84
2,2,커피,2,이디야,7.61,3.95
3,2,커피,3,엔제리너스,1.84,1.41
4,2,커피,4,카페베네,1.02,0.82
5,2,커피,5,커피빈,1.31,1.05
6,2,커피,6,탐앤탐스,1.27,0.93
7,2,커피,7,할리스,2.17,1.59
8,2,커피,8,파스쿠찌,1.43,1.15
9,2,커피,999,커피기타,68.13,72.97


### EXPORT csv FILE 

- UnicodeDecode Error : illegal multibyte sequence

p=p.replace('\\xa0', '')  ->  with open(filename, mode, encoding='utf8') as f:

In [21]:
export_data = pd.DataFrame([])
export_data= export_data.append(brand[['GROUP','GRUOP_NAME','CODE','CODE_NAME']].merge(make_stat_df(movie),how='inner',on='CODE_NAME'))
export_data= export_data.append(brand[['GROUP','GRUOP_NAME','CODE','CODE_NAME']].merge(make_stat_df(coffee),how='inner',on='CODE_NAME'))
export_data= export_data.append(brand[['GROUP','GRUOP_NAME','CODE','CODE_NAME']].merge(make_stat_df(beauty),how='inner',on='CODE_NAME'))
export_data.columns=[ '코드그룹','코드그룹명','코드','코드명','횟수비중(%)','금액비중(%)']
export_data.to_csv('data/CJ_BrandPct0621.csv')
print(export_data.shape)
export_data.head(31)

(31, 6)


,코드그룹,코드그룹명,코드,코드명,횟수비중(%),금액비중(%)
0,1,영화,0,CGV,57.30,52.67
1,1,영화,1,롯데시네마,42.33,46.95
2,1,영화,2,메가박스,0.22,0.15
3,1,영화,999,영화기타,0.16,0.24
0,2,커피,0,투썸플레이스,4.80,4.30
1,2,커피,1,스타벅스,10.42,11.84
2,2,커피,2,이디야,7.61,3.95
3,2,커피,3,엔제리너스,1.84,1.41
4,2,커피,4,카페베네,1.02,0.82
5,2,커피,5,커피빈,1.31,1.05
